In [17]:
from bs4 import BeautifulSoup
import requests
url='https://www.nationalgallery.org.uk/paintings/search-the-collection'
response = requests.get(url)
html = response.text
print(len(html))
soup = BeautifulSoup(html, 'html.parser')
items = soup.select('[class=image-item]')
print(len(items))

443116
0


In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd
import numpy as np
from tqdm import tqdm

In [24]:
driver = webdriver.Firefox()
url = 'https://www.nationalgallery.org.uk/paintings/search-the-collection'
driver.implicitly_wait(10)
driver.get(url)

In [25]:
# Identify "See More" button
btn = driver.find_elements(by = By.CLASS_NAME, value='see-more-link')
btn

[<selenium.webdriver.remote.webelement.WebElement (session="c84c3777-be0e-42d8-b0cd-b76e823282cc", element="6ff30b81-49fc-4980-93c9-578da56694d3")>]

In [13]:
driver.get(url)

In [27]:
# Click "See More" button until all the contents are loaded

while int(driver.find_elements(by= By.CLASS_NAME, value='x-viewed-count')[0].text) < int(driver.find_elements(by= By.CLASS_NAME, value='x-total-count')[0].text):
    print('Current : ', driver.find_elements(by= By.CLASS_NAME, value='x-viewed-count')[0].text)
    print('Total : ', driver.find_elements(by= By.CLASS_NAME, value='x-total-count')[0].text)
    
    btn[0].click()
    driver.implicitly_wait(3)

Current :  30
Total :  2620
Current :  30
Total :  2620
Current :  30
Total :  2620
Current :  60
Total :  2620
Current :  90
Total :  2620
Current :  90
Total :  2620
Current :  120
Total :  2620
Current :  150
Total :  2620
Current :  180
Total :  2620
Current :  180
Total :  2620
Current :  240
Total :  2620
Current :  270
Total :  2620
Current :  300
Total :  2620
Current :  330
Total :  2620
Current :  360
Total :  2620
Current :  390
Total :  2620
Current :  420
Total :  2620
Current :  450
Total :  2620
Current :  480
Total :  2620
Current :  510
Total :  2620
Current :  540
Total :  2620
Current :  570
Total :  2620
Current :  600
Total :  2620
Current :  630
Total :  2620
Current :  660
Total :  2620
Current :  690
Total :  2620
Current :  720
Total :  2620
Current :  750
Total :  2620
Current :  780
Total :  2620
Current :  810
Total :  2620
Current :  840
Total :  2620
Current :  870
Total :  2620
Current :  900
Total :  2620
Current :  930
Total :  2620
Current :  960
Total

In [28]:
items = driver.find_elements(by=By.CLASS_NAME, value='image-item')
len(items)

2730

In [29]:
# Test if all elements are loaded well by printing each piece's title

for item in items:
    # Find the title and get its text
    title = item.find_element(By.CLASS_NAME, 'title')
    print(title.text)


The Tower of Babel
The Market Cart
Landscape with Tobias and the Angel
The Virgin and Child
Cavalry making a Sortie from a Fort on a Hill
Queen Mariana of Spain in Mourning
The Adoration of the Kings
Cornelia and her Sons
The Trinity: Centre Pinnacle
The Outskirts of a Village, with a Horseman
Still Life with Lemons in a Wicker Basket
Ulysses after the Shipwreck
Triptych: Scenes from the Passion of Christ
A Courtyard in Rome
A White Horse, and an Old Man binding Faggots
A Dutch Ship coming to Anchor
Green Mountains
Portrait of a Man
An Extensive Landscape with a Hawking Party
The Cascade at Tivoli
Portrait of a Woman of the Hofer Family
Deauville Harbour
Monsieur Pivot on Horseback
Portrait of a Man in a Black Wig
Summer Morning
A Coastal Scene
An Interior with a Man offering an Oyster to a Woman
Saint Jerome in a Landscape
The Virgin and Child with Cherubim
The Virgin and Child
The Horrors of War
Portrait of Gerolamo (?) Barbarigo
Pharaoh with his Butler and Baker
The Madonna and Chil

In [30]:
# Initialize lists to store data
image_links = []
hyperlinks = []
titles = []
artists = []

n_total_items = len(items)

# Loop through the list of WebElement objects
for i, item in enumerate(items):
    # Find the image and get its source link
    image = item.find_element(By.TAG_NAME, 'img')
    image_links.append(image.get_attribute('src'))

    # Find the hyperlink and get its href
    hyperlink = item.find_element(By.TAG_NAME, 'a')
    hyperlinks.append(hyperlink.get_attribute('href'))

    # Find the title and get its text
    title = item.find_element(By.CLASS_NAME, 'title')
    titles.append(title.text)

    # Find the artist and get its text
    artist = item.find_element(By.CLASS_NAME, 'description')
    artists.append(artist.text)

    if i % 20 == 0:
        print(f"{i} items completed in {n_total_items}")


# Create a pandas DataFrame
df = pd.DataFrame({
    'Image Link': image_links,
    'Hyperlink': hyperlinks,
    'Title': titles,
    'Artist': artists
})

0 items completed in 2730
20 items completed in 2730
40 items completed in 2730
60 items completed in 2730
80 items completed in 2730
100 items completed in 2730
120 items completed in 2730
140 items completed in 2730
160 items completed in 2730
180 items completed in 2730
200 items completed in 2730
220 items completed in 2730
240 items completed in 2730
260 items completed in 2730
280 items completed in 2730
300 items completed in 2730
320 items completed in 2730
340 items completed in 2730
360 items completed in 2730
380 items completed in 2730
400 items completed in 2730
420 items completed in 2730
440 items completed in 2730
460 items completed in 2730
480 items completed in 2730
500 items completed in 2730
520 items completed in 2730
540 items completed in 2730
560 items completed in 2730
580 items completed in 2730
600 items completed in 2730
620 items completed in 2730
640 items completed in 2730
660 items completed in 2730
680 items completed in 2730
700 items completed in 273

In [32]:
# Create descriptions column to save the text data

df['Overview Description'] = np.zeros_like(titles).tolist()
df['In-depth Description'] = np.zeros_like(titles).tolist()

In [39]:
# Change to Edge browser
# Edge가 낫더라 ㅋㅋㅋ
driver = webdriver.Edge()

------ Working on piece No.0 ------


ElementClickInterceptedException: Message: element click intercepted: Element <button type="button" id="btn..." class="toggle-painting-overlay animated-underline" data-toggle-area="painting_description" aria-expanded="false" aria-controls="tabDescription">Description</button> is not clickable at point (223, 99). Other element would receive the click: <div id="CookiebotDialogScreen"></div>
  (Session info: MicrosoftEdge=113.0.1774.57)
Stacktrace:
0   msedgedriver                        0x000000010d9e71d8 msedgedriver + 5149144
1   msedgedriver                        0x000000010d9de7d3 msedgedriver + 5113811
2   msedgedriver                        0x000000010d570be6 msedgedriver + 469990
3   msedgedriver                        0x000000010d5bedd3 msedgedriver + 789971
4   msedgedriver                        0x000000010d5bc5ff msedgedriver + 779775
5   msedgedriver                        0x000000010d5b9a34 msedgedriver + 768564
6   msedgedriver                        0x000000010d5b8a8e mse

In [47]:
# Loop through the links dataframe to collect data

for i in range(1773,len(df)):
    driver.get(df.loc[i, 'Hyperlink'])
    driver.implicitly_wait(3)
    print(f'------ Working on piece No.{i} ------')

    # Identify "Description" button. No contents are loaded before the button is pressed.
    btn_desc = driver.find_elements(by=By.ID, value='btnDescription')

    if len(btn_desc) == 0:
        # If there's no description, skip the piece and continue to the next piece.
        print('No description is available. Continuing to next piece.')
        continue

    # Click "Description" button and collect overview description data.
    btn_desc[0].click()
    element_short = driver.find_elements(by = By.CLASS_NAME, value = 'short-text')
    if len(element_short) == 0:
        pass
    else:
        # Find all paragraphs in the element
        paragraphs_short = element_short[0].find_elements(By.CLASS_NAME, 'dokuwiki_paragraph')
        # Extract the text from each paragraph and join them with a newline
        description_short = '\n'.join([p.text for p in paragraphs_short])
        df.loc[i, 'Overview Description'] = description_short
        print(f'Overview description recorded : {description_short[:20]}...')

    # Identify "In-depth Descrption" button. In depth description is not loaded until the button is pressed.
    btn_long = driver.find_elements(by=By.ID, value='t-long-text')
    
    if len(btn_long) == 0:
        # If there's no in depth descrption, continue to the next piece.
        pass
    else:
        print('In depth description detected.')
        btn_long[0].click()
        element_long = driver.find_elements(by = By.CLASS_NAME, value = 'long-text')
        # Find all paragraphs in the element
        paragraphs_long = element_long[0].find_elements(By.CLASS_NAME, 'dokuwiki_paragraph')
        
        # Extract the text from each paragraph and join them with a newline
        description_long = '\n'.join([p.text for p in paragraphs_long])
        
        # Save the description in the 'In-depth Description' column
        df.loc[i, 'In-depth Description'] = description_long

        print(f'In-depth description recorded : {description_long[:20]}...')


------ Working on piece No.1773 ------
Overview description recorded : , a fourteenth-centu...
In depth description detected.
In-depth description recorded : This is the third an...
------ Working on piece No.1774 ------
Overview description recorded : In the courtyard of ...
------ Working on piece No.1775 ------
Overview description recorded : This painting – an e...
In depth description detected.
In-depth description recorded : This painting – an e...
------ Working on piece No.1776 ------
Overview description recorded : Andrea Gritti (1455–...
In depth description detected.
In-depth description recorded : Andrea Gritti (1455–...
------ Working on piece No.1777 ------
Overview description recorded : We don't know the id...
In depth description detected.
In-depth description recorded : Jakob Seisenegger wa...
------ Working on piece No.1778 ------
Overview description recorded : ...
------ Working on piece No.1779 ------
Overview description recorded : This oil sketch was ...
In dept

In [51]:
# Assign unique ids for each piece.

ID = np.arange(len(titles)).tolist()
df['ID'] = ID

In [55]:
print(df.columns)

Index(['Image Link', 'Hyperlink', 'Title', 'Artist', 'Overview Description',
       'In-depth Description', 'ID'],
      dtype='object')


In [56]:
# Save the data as pickle.
df.to_pickle('./NationalGallery.pkl')

In [57]:
driver.close()

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: MicrosoftEdge=113.0.1774.57)
Stacktrace:
0   msedgedriver                        0x000000010d9e71d8 msedgedriver + 5149144
1   msedgedriver                        0x000000010d9de7d3 msedgedriver + 5113811
2   msedgedriver                        0x000000010d570be6 msedgedriver + 469990
3   msedgedriver                        0x000000010d555c28 msedgedriver + 359464
4   msedgedriver                        0x000000010d5572cc msedgedriver + 365260
5   msedgedriver                        0x000000010d557224 msedgedriver + 365092
6   msedgedriver                        0x000000010d544d8e msedgedriver + 290190
7   msedgedriver                        0x000000010d545cb6 msedgedriver + 294070
8   msedgedriver                        0x000000010d5e794f msedgedriver + 956751
9   msedgedriver                        0x000000010d5db518 m